In [36]:
import torch
from torchvision import io, transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm

In [37]:
class mnist_dataset(Dataset):
    def __init__(self, root_dir, train=True, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.data = []
        self.labels = []
        
        folder = 'train' if train else 'test'
        folder_path = os.path.join(root_dir, folder)
        
        for label in os.listdir(folder_path):
            label_path = os.path.join(folder_path, label)
            
            for img_name in os.listdir(label_path):
                img_path = os.path.join(label_path, img_name)
                self.data.append(img_path)
                self.labels.append(int(label))
                
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        img_path = self.data[index]
        label = self.labels[index]
        img = Image.open(img_path)
        
        if self.transform:
            img = self.transform(img)
            
        return img, label

In [38]:
transform = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = mnist_dataset(root_dir='mnist', train=True, transform=transform)
test_dataset = mnist_dataset(root_dir='mnist', train=False, transform=transform)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [136]:

class cnn(torch.nn.Module):
    def __init__(self):
        super(cnn, self).__init__()
        self.dropout_prob = 0.5
        
        # Layer 1
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=2),
            torch.nn.ReLU(),
            torch.nn.Dropout2d(self.dropout_prob)
        )
        
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=32, out_channels=16, kernel_size=3, stride=2, padding=1),
            torch.nn.ReLU(),
            torch.nn.Dropout2d(self.dropout_prob)
        )
        
        self.layer3 = torch.nn.Sequential(
            torch.nn.Flatten(start_dim=1, end_dim=-1),
            torch.nn.Linear(16*14*14, 32),
            torch.nn.Linear(32, 10),
        )
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x


In [137]:
model = cnn()
model = model.to('cuda')
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [140]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in tqdm(enumerate(train_dataloader, 0)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        inputs = inputs.to('cuda')
        labels = labels.to('cuda')
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

938it [00:12, 75.55it/s]
938it [00:12, 76.26it/s]
938it [00:12, 76.46it/s]
938it [00:12, 75.09it/s]
938it [00:12, 74.60it/s]
938it [00:12, 74.78it/s]
938it [00:12, 75.04it/s]
938it [00:12, 73.90it/s]
938it [00:12, 74.17it/s]
938it [00:12, 74.27it/s]

Finished Training


In [141]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in tqdm(test_dataloader):
        images, labels = data
        images = images.to('cuda')
        labels = labels.to('cuda')
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

100%|██████████| 157/157 [00:01<00:00, 79.69it/s]

Accuracy of the network on the 10000 test images: 92 %
